In [150]:
import pandas as pd
import numpy as np
import os

os.environ["PYTHONWARNINGS"] = "ignore"

In [151]:
df = pd.read_json("DATASET_FINAL.json")
# df = df.drop(["budget", "duration"], axis = 1)
df = df.drop(["duration"], axis = 1)
df["country"] = df["country"].str.strip()


In [152]:
df.isna().sum()

fr_title               0
released_year          0
directors              0
writer                 0
distribution           0
country                0
budget              1902
category               0
released_date          0
classification         0
weekly_entrances       0
duration_minutes       0
actor_1                0
actor_2                0
actor_3                0
dtype: int64

In [153]:
df.dtypes

fr_title             object
released_year         int64
directors            object
writer               object
distribution         object
country              object
budget              float64
category             object
released_date        object
classification       object
weekly_entrances      int64
duration_minutes      int64
actor_1              object
actor_2              object
actor_3              object
dtype: object

In [154]:
# conversion de la colonne released_date en datetime
df["released_date"] = df["released_date"].str.strip()
df["released_date"] = pd.to_datetime(df["released_date"], format='%d/%m/%Y')


In [155]:
# df[df["budget"].isna()]
# df[(df["country"] == "Etats-Unis") & (df["budget"].isna())].head(50)


In [156]:
df.dtypes

fr_title                    object
released_year                int64
directors                   object
writer                      object
distribution                object
country                     object
budget                     float64
category                    object
released_date       datetime64[ns]
classification              object
weekly_entrances             int64
duration_minutes             int64
actor_1                     object
actor_2                     object
actor_3                     object
dtype: object

In [157]:
print("directors : ", df["directors"].unique().shape[0])
print("writer : ", df["writer"].unique().shape[0])
print("distribution : ", df["distribution"].unique().shape[0])
print("country : ", df["country"].unique().shape[0])
print("category : ", df["category"].unique().shape[0])
print("classification : ", df["classification"].unique().shape[0])
print("actor_1 : ", df["actor_1"].unique().shape[0])
print("actor_2 : ", df["actor_2"].unique().shape[0])
print("actor_3 : ", df["actor_3"].unique().shape[0])


directors :  3441
writer :  4286
distribution :  1126
country :  73
category :  23
classification :  7
actor_1 :  2917
actor_2 :  3650
actor_3 :  4261


In [158]:
# df = df.dropna(subset=["budget"], axis=0)

# df[df["country"] == "Japon"]["budget"].median()


In [159]:
print(df.shape)
df.isna().sum()
df.dtypes
df["directors"] = df["directors"].rename()

(7251, 15)


# traitements : 

#### country : 
One hot encoding
regrouper par fréquence (les pays qui produisent le plus - prendre un nombre de film comme seuil - seuil a déterminer par une EDA)
    -> garder les 10 pays qui produisent le plus et le reste le mettre dans autres

#### category : 
one hot encoding

#### classification : 
One hot encoding

#### director, writer, distribution, actors : 
nombre d'occurences (fréquence)
popularité basée sur les entrées passées
classement des plus connus (depuis une liste externe)
    - top 100 ?
    - top 200 ?
    - top 500 ?

#### release_date : 
    - Vacances scolaires
    - saisons
    - festivals
    - Année stratégique
        - is_covid_year = df["released_year"].isin([2020, 2021])
        - post_streaming = df["released_year"] >= 2012
        - release_decade = df["released_year"] // 10 * 10

    - apparition des plateformes de streaming
    - Semaine spéciale
        - is_christmas_week → sortie entre 24 déc - 31 déc
        - is_new_year_week → 1er janvier
        - is_summer_release → juillet / août
        - is_back_to_school → semaine 36-38
        - is_award_season → jan-fév (sorties Oscar / César)
        - is_festival_period → mai pour Cannes, septembre pour TIFF/Venise

#### budget : 
    - supprimer les budgets extraits de imdb ?
    - créer une colonne budget disponible (oui ou non) ?
    - Créez des catégories de budget (petit/moyen/gros) quand l'information est disponible ?
    - Utilisez "inconnu" comme catégorie supplémentaire ?
    - supprimer les lignes ou les budgets sont nuls ?

###### country          OK
###### category         OK
###### classification   OK
###### director         OK
###### writer           OK
###### distribution     OK
###### actors           OK
###### release_date     OK
###### release_year     OK
###### budget

In [160]:
print(df[df["actor_1"] == "no_actor"].shape)
print(df[df["actor_2"] == "no_actor"].shape)
print(df[df["actor_3"] == "no_actor"].shape)

(72, 15)
(105, 15)
(128, 15)


In [161]:
# classification, country, category - one hot encoding ou get dummies

df = pd.get_dummies(df, columns=["classification", "category", "country"], drop_first=True)


In [162]:
# director

df_directors = df.groupby('directors')['weekly_entrances'].mean().reset_index()

df_directors_low = df_directors[df_directors["weekly_entrances"] < 100000]
df_directors_middle = df_directors[(df_directors["weekly_entrances"] >= 100000) & (df_directors["weekly_entrances"] <= 500000)]
df_directors_high = df_directors[df_directors["weekly_entrances"] > 500000]

df["top_director_low"] = df["directors"].apply(lambda x: 1 if x in df_directors_low["directors"].to_list() else 0)
df["top_director_middle"] = df["directors"].apply(lambda x: 1 if x in df_directors_middle["directors"].to_list() else 0)
df["top_director_high"] = df["directors"].apply(lambda x: 1 if x in df_directors_high["directors"].to_list() else 0)

# df['top_director'] = df['directors'].apply(lambda x : 1 if x in(df_directors['directors'].to_list()) else 0)


In [163]:
# writer

df_writer = df.groupby('writer')['weekly_entrances'].mean().reset_index()

df_writer_low = df_writer[df_writer["weekly_entrances"] < 100000]
df_writer_middle = df_writer[(df_writer["weekly_entrances"] >= 100000) & (df_writer["weekly_entrances"] <= 500000)]
df_writer_high = df_writer[df_writer["weekly_entrances"] > 500000]

df["top_writer_low"] = df["writer"].apply(lambda x: 1 if x in df_writer_low["writer"].to_list() else 0)
df["top_writer_middle"] = df["writer"].apply(lambda x: 1 if x in df_writer_middle["writer"].to_list() else 0)
df["top_writer_high"] = df["writer"].apply(lambda x: 1 if x in df_writer_high["writer"].to_list() else 0)



In [164]:
# distribution

df_distribution = df.groupby('distribution')['weekly_entrances'].mean().reset_index()

df_distribution_low = df_distribution[df_distribution["weekly_entrances"] < 100000]
df_distribution_middle = df_distribution[(df_distribution["weekly_entrances"] >= 100000) & (df_distribution["weekly_entrances"] <= 500000)]
df_distribution_high = df_distribution[df_distribution["weekly_entrances"] > 500000]

df["top_distribution_low"] = df["distribution"].apply(lambda x: 1 if x in df_distribution_low["distribution"].to_list() else 0)
df["top_distribution_middle"] = df["distribution"].apply(lambda x: 1 if x in df_distribution_middle["distribution"].to_list() else 0)
df["top_distribution_high"] = df["distribution"].apply(lambda x: 1 if x in df_distribution_high["distribution"].to_list() else 0)


In [165]:
# actor_1

df_actor_1 = df.groupby('actor_1')['weekly_entrances'].mean().reset_index()

df_actor_1_low = df_actor_1[df_actor_1["weekly_entrances"] < 100000]
df_actor_1_middle = df_actor_1[(df_actor_1["weekly_entrances"] >= 100000) & (df_actor_1["weekly_entrances"] <= 500000)]
df_actor_1_high = df_actor_1[df_actor_1["weekly_entrances"] > 500000]

df["top_actor_1_low"] = df["actor_1"].apply(lambda x: 1 if x in df_actor_1_low["actor_1"].to_list() and x != "no_actor" else 0)

df["top_actor_1_middle"] = df["actor_1"].apply(lambda x: 1 if x in df_actor_1_middle["actor_1"].to_list() and x != "no_actor" else 0)
df["top_actor_1_high"] = df["actor_1"].apply(lambda x: 1 if x in df_actor_1_high["actor_1"].to_list() and x != "no_actor" else 0)



In [166]:
# actor_2

df_actor_2 = df.groupby('actor_2')['weekly_entrances'].mean().reset_index()

df_actor_2_low = df_actor_2[df_actor_2["weekly_entrances"] < 100000]
df_actor_2_middle = df_actor_2[(df_actor_2["weekly_entrances"] >= 100000) & (df_actor_2["weekly_entrances"] <= 500000)]
df_actor_2_high = df_actor_2[df_actor_2["weekly_entrances"] > 500000]

df["top_actor_2_low"] = df["actor_2"].apply(lambda x: 1 if x in df_actor_2_low["actor_2"].to_list() else 0)
df["top_actor_2_middle"] = df["actor_2"].apply(lambda x: 1 if x in df_actor_2_middle["actor_2"].to_list() else 0)
df["top_actor_2_high"] = df["actor_2"].apply(lambda x: 1 if x in df_actor_2_high["actor_2"].to_list() else 0)



In [167]:
# actor_3

df_actor_3 = df.groupby('actor_3')['weekly_entrances'].mean().reset_index()

df_actor_3_low = df_actor_3[df_actor_3["weekly_entrances"] < 100000]
df_actor_3_middle = df_actor_3[(df_actor_3["weekly_entrances"] >= 100000) & (df_actor_3["weekly_entrances"] <= 500000)]
df_actor_3_high = df_actor_3[df_actor_3["weekly_entrances"] > 500000]

df["top_actor_3_low"] = df["actor_3"].apply(lambda x: 1 if x in df_actor_3_low["actor_3"].to_list() else 0)
df["top_actor_3_middle"] = df["actor_3"].apply(lambda x: 1 if x in df_actor_3_middle["actor_3"].to_list() else 0)
df["top_actor_3_high"] = df["actor_3"].apply(lambda x: 1 if x in df_actor_3_high["actor_3"].to_list() else 0)



In [168]:
# release_date

"""
#### release_date : 
    - mois de vacances scolaires (été)
    - saisons
    - Année stratégique
        - is_covid_year = df["released_year"].isin([2020, 2021])
        - post_streaming = df["released_year"] >= 2012

    - Semaine spéciale
        - is_christmas_week → sortie entre 24 déc - 31 déc
        - is_new_year_week → 1er janvier
            - noel + new year : 20/12 au 05/01
        - is_award_season → fev - mars (sorties Oscar / César) 
"""

df["summer"] = df["released_date"].apply(lambda x: 1 if ((x.month == 6 and x.day >= 21) or x.month in [7, 8] or (x.month == 9 and x.day < 22)) else 0)
df["automn"] = df["released_date"].apply(lambda x: 1 if ((x.month == 9 and x.day >= 22) or x.month in [10, 11] or (x.month == 12 and x.day < 21)) else 0)
df["winter"] = df["released_date"].apply(lambda x: 1 if ((x.month == 12 and x.day >= 21) or x.month in [1, 2] or (x.month == 3 and x.day < 20)) else 0)
df["spring"] = df["released_date"].apply(lambda x: 1 if ((x.month == 3 and x.day >= 21) or x.month in [4, 5] or (x.month == 6 and x.day < 21)) else 0)

df["is_covid"] = df["released_date"].apply(lambda x: 1 if (
    (x >= pd.to_datetime("2020-03-17") and x <= pd.to_datetime("2020-05-11")) or
    (x >= pd.to_datetime("2020-10-30") and x <= pd.to_datetime("2020-12-15")) or
    (x >= pd.to_datetime("2021-04-03") and x <= pd.to_datetime("2021-05-03"))
) else 0)

df["post_streaming"] = df["released_date"].apply(lambda x: 1 if x >= pd.to_datetime("2014-09-15") else 0)

df["summer_holidays"] = df["released_date"].apply(lambda x: 1 if x.month >= 7 or (x.month <= 9 and x.day < 10) else 0)

df["christmas_period"] = df["released_date"].apply(lambda x: 1 if (x.month == 12 and x.day >= 20) or (x.month == 1 and x.day <= 5) else 0)

df["is_award_season"] = df["released_date"].apply(lambda x: 1 if (x.month == 2 or (x.month == 3 and x.day <= 10)) else 0)



In [169]:
# autres transformations 

# Mois et transformations cycliques (capture mieux les patterns saisonniers)
df['month'] = df['released_date'].dt.month
df['cos_month'] = np.cos(2 * np.pi * df['month'] / 12)
df['sin_month'] = np.sin(2 * np.pi * df['month'] / 12)


In [170]:
# from sklearn.model_selection import train_test_split

# # Créer des bins plus détaillés pour une meilleure stratification
# df['success_level'] = pd.cut(df['weekly_entrances'],
#                             bins=[0, 50000, 200000, 500000, 1000000, float('inf')],
#                             labels=[0, 1, 2, 3, 4])

# df = df.dropna(subset=["budget"])
# X = df.drop("weekly_entrances", axis = 1)
# y = df["weekly_entrances"]


# # Stratifier sur ces niveaux
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.15, random_state=42, stratify=df['success_level']
# )

In [171]:
df.shape

(7251, 142)

# Modélisation

In [172]:
df_initial = df.copy()

In [173]:
# df = df_initial.select_dtypes(exclude="object")
df = df_initial.select_dtypes(exclude=["object", "datetime"]).drop(["released_year"], axis = 1)

In [174]:
list(df.columns)

['budget',
 'weekly_entrances',
 'duration_minutes',
 'classification_Interdit - 12 ans',
 'classification_Interdit - 16 ans',
 'classification_Interdit - 18 ans',
 'classification_Interdit - 6 ans',
 'classification_Interdit - 8 ans',
 'classification_Tout public',
 'category_Adaptation Livre',
 'category_Animation',
 'category_Aventure - Action',
 'category_Catastrophe',
 'category_Comicbook',
 'category_Comédie',
 'category_Comédie dramatique',
 'category_Court-métrage',
 'category_Documentaire',
 'category_Drame',
 'category_Fantasy',
 'category_Film familial',
 'category_Guerre',
 'category_Horreur',
 'category_Musical',
 'category_Parodie',
 'category_Péplum',
 'category_Romance',
 'category_Science Fiction',
 'category_Thriller',
 'category_Western',
 'category_X - Erotique',
 'country_Afghanistan',
 'country_Afrique du Sud',
 'country_Algérie',
 'country_Allemagne',
 'country_Arabie Saoudite',
 'country_Argentine',
 'country_Australie',
 'country_Autriche',
 'country_Belgique',

In [175]:
df.dtypes.unique()

array([dtype('float64'), dtype('int64'), dtype('bool'), dtype('int32')],
      dtype=object)

In [176]:
df.head(2)

,budget,weekly_entrances,duration_minutes,classification_Interdit - 12 ans,classification_Interdit - 16 ans,classification_Interdit - 18 ans,classification_Interdit - 6 ans,classification_Interdit - 8 ans,classification_Tout public,category_Adaptation Livre,...,winter,spring,is_covid,post_streaming,summer_holidays,christmas_period,is_award_season,month,cos_month,sin_month
0,27800000.0,2429906,107,False,False,False,False,False,True,False,...,1,0,0,1,0,0,1,2,0.5,0.866025
1,24434009.0,2587056,108,False,False,False,False,False,False,False,...,1,0,0,0,1,0,1,2,0.5,0.866025


## premier modele

In [177]:
from sklearn.model_selection import train_test_split

# Créer des bins plus détaillés pour une meilleure stratification
df['success_level'] = pd.cut(df['weekly_entrances'],
                            bins=[0, 50000, 200000, 500000, 1000000, float('inf')],
                            labels=[0, 1, 2, 3, 4])

# df = df.dropna(subset=["budget"])
df = df.drop(["budget"], axis = 1)
X = df.drop("weekly_entrances", axis = 1)
y = df["weekly_entrances"]


# Stratifier sur ces niveaux
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=df['success_level']
)

In [178]:

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error

def metriques(model, X_test, y_test) : 
    
    pred_y = model.predict(X_test)
    mse = mean_squared_error(y_test, pred_y)
    mae = mean_absolute_error(y_test, pred_y)
    r2 = r2_score(y_test, pred_y)
    med_ae = median_absolute_error(y_test, pred_y)
    
    
    print(f"MSE : {mse}")
    print(f"RMSE : {np.sqrt(mse)}")
    print(f"MAE : {mae}")
    print(f"R2 : {r2}")
    print(f"MedAE : {med_ae}")
    
    return mse, mae, r2, med_ae


def coefficients(model, X) : 
    coefs = model.coef_
    noms_cols = X.columns
    coefs_df = pd.DataFrame({"variables" : noms_cols, "coef" : coefs})
    print("coefficients des variables")
    print(coefs_df)


In [179]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

cat_features = [i for i, col in enumerate(X_train.columns) if X_train[col].dtype.name == 'category']

models = {
    "model_lr": LinearRegression(),
    "model_lasso": Lasso(alpha=0.0001),
    "model_ridge": Ridge(alpha=0.7),
    "model_rf": RandomForestRegressor(n_estimators=100, random_state=42),
    "model_gbr": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "model_xgb": XGBRegressor(n_estimators=100, random_state=42, enable_categorical=True),
    "model_lgbm": LGBMRegressor(n_estimators=100, random_state=42),
    "model_catboost": CatBoostRegressor(iterations=300, random_seed=42, depth=6, verbose=0), 
    "model_svr": SVR(kernel='rbf', C=1.0, epsilon=0.1),
    "model_knn": KNeighborsRegressor(n_neighbors=5),
    "model_dt": DecisionTreeRegressor(max_depth=10, random_state=42),
    "model_ada": AdaBoostRegressor(random_state=42)
}


In [180]:
# Entraînement avec pondération des échantillons
sample_weights = np.ones(len(y_train))
success_mask = y_train > 500000
sample_weights[success_mask] = 5  # Pondération 3x plus importante

for m in models.keys():
    print(f"entrainement du modele {m} : \n")

    if m == "model_catboost":
        models[m].fit(X_train, y_train, cat_features=cat_features)
    else:
        models[m].fit(X_train, y_train)

    print(f"résultats du model {m} \n")
    metriques(models[m], X_test=X_test, y_test=y_test)
    print("\n----------------- \n")    



entrainement du modele model_lr : 

résultats du model model_lr 

MSE : 36229959310.388145
RMSE : 190341.69094128627
MAE : 97107.57588855366
R2 : 0.7404778166321538
MedAE : 65883.27174741557

----------------- 

entrainement du modele model_lasso : 



/home/malek/BRIEFS DEV IA/9. MLRecap/MLrecap_Model/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.633e+13, tolerance: 7.168e+10
  model = cd_fast.enet_coordinate_descent(


résultats du model model_lasso 

MSE : 36217259904.54655
RMSE : 190308.32852123564
MAE : 97033.3214371717
R2 : 0.7405687849245345
MedAE : 65659.30152105124

----------------- 

entrainement du modele model_ridge : 

résultats du model model_ridge 

MSE : 36188372925.33281
RMSE : 190232.41817664204
MAE : 96859.6292001388
R2 : 0.7407757079258059
MedAE : 65206.66626538752

----------------- 

entrainement du modele model_rf : 

résultats du model model_rf 

MSE : 17777576701.019444
RMSE : 133332.5792933574
MAE : 49302.41833336981
R2 : 0.8726557907252452
MedAE : 18587.025

----------------- 

entrainement du modele model_gbr : 

résultats du model model_gbr 

MSE : 19540249191.247856
RMSE : 139786.44137128556
MAE : 49198.263182752125
R2 : 0.860029427849498
MedAE : 20146.104215153515

----------------- 

entrainement du modele model_xgb : 

résultats du model model_xgb 

MSE : 19060197376.0
RMSE : 138058.67367174002
MAE : 50378.66796875
R2 : 0.8634681105613708
MedAE : 18432.7578125

-------

In [181]:
# catboost TOP résultats aussi

from catboost import CatBoostRegressor

model_catboost = CatBoostRegressor(iterations=100, random_seed=42, depth=6, verbose=0)


model_catboost.fit(X_train, y_train, cat_features=cat_features)
print("Résultats CatBoost :")
metriques(model_catboost, X_test, y_test)

Résultats CatBoost :
MSE : 15980268168.587412
RMSE : 126413.08543259045
MAE : 48041.17937082459
R2 : 0.8855302582488308
MedAE : 18360.994160578084


(15980268168.587412,
 48041.17937082459,
 0.8855302582488308,
 np.float64(18360.994160578084))

In [195]:
catboost_params = [
    {  # Pour grow_policy=SymmetricTree/Depthwise
        'grow_policy': ['SymmetricTree', 'Depthwise'],
        'iterations': [150, 200, 250, 300],
        'learning_rate': [0.03, 0.05, 0.07, 0.1],
        'depth': [5, 6, 7, 8],
        'min_data_in_leaf': [1, 5, 10, 20],
        'l2_leaf_reg': [1, 3, 5, 7],
        'random_strength': [0.5, 1.0, 1.5]
    },
    {  # Pour grow_policy=Lossguide
        'grow_policy': ['Lossguide'],
        'max_leaves': [31, 55, 80],
        'iterations': [150, 200, 250, 300],
        'learning_rate': [0.03, 0.05, 0.07, 0.1],
        'min_data_in_leaf': [1, 5, 10, 20],
        'l2_leaf_reg': [1, 3, 5, 7],
        'random_strength': [0.5, 1.0, 1.5]
    }
]


In [196]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    CatBoostRegressor(task_type='GPU', devices='0', verbose=0),
    param_distributions=catboost_params[1],
    n_iter=50,  # Nombre d'itérations
    cv=5,
    scoring='neg_root_mean_squared_error',
    random_state=42,
    n_jobs=1  # Important pour GPU
)

random_search.fit(X_train, y_train, cat_features=cat_features)
print(f"Meilleurs paramètres: {random_search.best_params_}")

ValueError: 
All the 250 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
250 fits failed with the following error:
Traceback (most recent call last):
  File "/home/malek/BRIEFS DEV IA/9. MLRecap/MLrecap_Model/.venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/malek/BRIEFS DEV IA/9. MLRecap/MLrecap_Model/.venv/lib/python3.11/site-packages/catboost/core.py", line 5873, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, graph, sample_weight, None, None, None, None, baseline,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/malek/BRIEF

In [184]:
# from sklearn.model_selection import GridSearchCV

# # Après avoir trouvé les meilleurs paramètres avec la recherche grossière
# best_params = random_search.best_params_

# # Créer une grille fine autour des meilleurs paramètres
# fine_params = {
#     'iterations': [best_params['iterations'] - 50, best_params['iterations'], best_params['iterations'] + 50],
#     'learning_rate': [best_params['learning_rate'] * 0.8, best_params['learning_rate'], best_params['learning_rate'] * 1.2],
#     'depth': [max(1, best_params['depth'] - 1), best_params['depth'], best_params['depth'] + 1],
#     'l2_leaf_reg': [max(1, best_params['l2_leaf_reg'] - 1), best_params['l2_leaf_reg'], best_params['l2_leaf_reg'] + 1]
# }

# grid_search = GridSearchCV(
#     CatBoostRegressor(verbose=0, **{k: v for k, v in best_params.items() if k not in fine_params}),
#     param_grid=fine_params,
#     cv=5,
#     scoring='neg_root_mean_squared_error',
#     n_jobs=1
# )
# grid_search.fit(X_train, y_train)
# print(f"Paramètres optimaux: {grid_search.best_params_}")

In [185]:
# import matplotlib.pyplot as plt

# # Créer le modèle final avec les meilleurs paramètres
# best_params = grid_search.best_params_
# print("Meilleurs paramètres trouvés:", best_params)

# # Créer et entraîner le modèle final
# final_model = CatBoostRegressor(
#     **best_params,
#     task_type='GPU',
#     devices='0',
#     verbose=0
# )

# # Entraîner sur l'ensemble des données d'entraînement
# final_model.fit(X_train, y_train)

# # Évaluer sur l'ensemble de test
# def metriques_detaillees(model, X_test, y_test):
#     # Prédictions
#     pred_y = model.predict(X_test)

#     # Métriques globales
#     mse = mean_squared_error(y_test, pred_y)
#     rmse = np.sqrt(mse)
#     mae = mean_absolute_error(y_test, pred_y)
#     r2 = r2_score(y_test, pred_y)
#     med_ae = median_absolute_error(y_test, pred_y)

#     print("=== Métriques globales ===")
#     print(f"MSE : {mse:.2f}")
#     print(f"RMSE : {rmse:.2f}")
#     print(f"MAE : {mae:.2f}")
#     print(f"R2 : {r2:.4f}")
#     print(f"MedAE : {med_ae:.2f}")

#     # Métriques par segment d'entrées
#     segments = [
#         (0, 50000, "Petit succès (<50K)"),
#         (50000, 200000, "Succès moyen (50K-200K)"),
#         (200000, 500000, "Grand succès (200K-500K)"),
#         (500000, float('inf'), "Blockbuster (>500K)")
#     ]

#     print("\n=== Métriques par segment ===")
#     for low, high, label in segments:
#         mask = (y_test >= low) & (y_test < high)
#         if mask.sum() > 0:
#             segment_rmse = np.sqrt(mean_squared_error(y_test[mask], pred_y[mask]))
#             segment_mae = mean_absolute_error(y_test[mask], pred_y[mask])
#             segment_r2 = r2_score(y_test[mask], pred_y[mask]) if mask.sum() > 1 else float('nan')

#             print(f"\n{label} (n={mask.sum()}):")
#             print(f"  RMSE : {segment_rmse:.2f}")
#             print(f"  MAE : {segment_mae:.2f}")
#             print(f"  R2 : {segment_r2:.4f}")

#     # Visualisation des erreurs
#     error = y_test - pred_y
#     plt.figure(figsize=(10, 6))
#     plt.scatter(y_test, error, alpha=0.5)
#     plt.axhline(y=0, color='r', linestyle='-')
#     plt.xlabel('Entrées réelles')
#     plt.ylabel('Erreur (Réel - Prédit)')
#     plt.title('Erreurs de prédiction par rapport aux entrées réelles')
#     plt.grid(True)
#     plt.show()

#     # Prédictions vs réalité
#     plt.figure(figsize=(10, 6))
#     plt.scatter(y_test, pred_y, alpha=0.5)
#     plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
#     plt.xlabel('Entrées réelles')
#     plt.ylabel('Entrées prédites')
#     plt.title('Prédictions vs Réalité')
#     plt.grid(True)
#     plt.show()

#     return mse, rmse, mae, r2, med_ae

# # Évaluer le modèle final
# print("\nPerformances du modèle final:")
# metriques_detaillees(final_model, X_test, y_test)

# # Importance des features
# feature_importance = pd.DataFrame({
#     'Feature': X_train.columns,
#     'Importance': final_model.feature_importances_
# }).sort_values('Importance', ascending=False)

# print("\nTop 20 features les plus importantes:")
# print(feature_importance.head(20))

# # Sauvegarder le modèle
# final_model.save_model('catboost_final_model.cbm')
# print("\nModèle sauvegardé sous 'catboost_final_model.cbm'")

In [186]:
# from sklearn.model_selection import GridSearchCV
# # import cupy as cp

# # Paramètres pour XGBoost GPU
# xgb_params = {
#     'n_estimators': [100, 200, 500],
#     'learning_rate': [0.05, 0.1, 0.2],
#     'max_depth': [4, 6, 8],
#     'subsample': [0.8, 0.9, 1.0],
#     'colsample_bytree': [0.8, 0.9, 1.0]
# }

# # GridSearchCV avec XGBoost GPU
# xgb_grid = GridSearchCV(
#     XGBRegressor(
#         random_state=42,
#         enable_categorical=True,
#         device='cuda',  # Utiliser GPU pour l'entraînement
#     ),
#     xgb_params,
#     cv=5,
#     scoring='neg_root_mean_squared_error',
#     n_jobs=1  # Important: utilisez 1 avec GPU pour éviter les conflits
# )

# # Entraînement avec pondération des échantillons
# sample_weights = np.ones(len(y_train))
# success_mask = y_train > 500000
# sample_weights[success_mask] = 5  # Pondération 3x plus importante

# # X_gpu = cp.asarray(X)  # Conversion CPU → GPU
# xgb_grid.fit(X_train, y_train, sample_weight=sample_weights)

# print(f"Meilleurs paramètres XGBoost GPU: {xgb_grid.best_params_}")
# print(f"Meilleur RMSE: {-xgb_grid.best_score_:.2f}")